# NVTabular on the GPU from an Internet enabled notebook

In [ ]:
pip install NVTabular

# NVTabular on the GPU without Internet

We unfortunately have to be a bit tricky here. Kaggle automatically unpacks archives in datasets, and Python packages are just that, archives. I zipped everything in order to avoid this problem, but Kaggle would still unarchive my main archive and continue unarchiving things.

In order to work around the issue I mention above I removed the file extension on my archive. Below we are able to extract it regardless.

In [ ]:
!unzip -q ../input/nvtabular-with-dependencies/nvt_all -d nvtabular-with-dependencies

In [ ]:
!pip install --no-index --find-links ./nvtabular-with-dependencies nvtabular

In [ ]:
# The commands I used to download the dependencies.

# !mkdir nvtabular_with_dependencies && cd nvtabular_with_dependencies && pip download nvtabular wheel pybind11
# !cd nvtabular_with_dependencies  && zip -r nvt_all.zip .

# from IPython.lib.display import FileLink
# FileLink('nvtabular_with_dependencies/nvt_all.zip')

# Check that everything worked and NVTabular runs on the GPU

In [ ]:
import nvtabular as nvt
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(data={'cats': np.random.randint(0, 100_000, 300_000_000), 'values': np.random.randn(300_000_000)})
ds = nvt.Dataset(df, cpu=False, npartitions=10)

out = ['cats', 'values'] >> nvt.ops.Groupby('cats', aggs=['min', 'std', 'last', 'max', 'mean'])
cats_hashed = ['cats'] >> nvt.ops.HashBucket(10)

wf = nvt.Workflow(out + cats_hashed)
o = wf.fit_transform(ds)